# Semantic Text Similarity

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# Requisites
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
import numpy as np
from typing import Tuple, List, Optional
from gensim.models import TfidfModel
from scipy import spatial

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Carreguem el model de Word2Vec preentrenat
from gensim.models import Word2Vec
WORD_EMBEDDING_FILE = '/content/drive/MyDrive/Pràctica 4 PLH/models Word2Vec/catalan_word2vec_1GB.model'
word2vec_model = Word2Vec.load(WORD_EMBEDDING_FILE)
vector_size = word2vec_model.vector_size

In [ ]:
from gensim.models import KeyedVectors  # Importamos KeyedVectors para manejar el modelo
# Configuración para decidir si usar mmap o no
USE_MMAP = False
if USE_MMAP:
    # Define el path donde quieres guardar el modelo mapeado
    MMAP_PATH = 'word2vec_catalan.model.bin'
    # Guarda el modelo en el formato adecuado para mmap
    word2vec_model.wv.save(MMAP_PATH)
    # Carga el modelo usando mmap para un acceso más eficiente a los vectores
    wv_model = KeyedVectors.load(MMAP_PATH, mmap='r')
else:
    # Si no estás usando mmap, simplemente carga el modelo como normalmente lo harías
    WORD_EMBEDDING_FILE = '/content/drive/MyDrive/Pràctica 4 PLH/models Word2Vec/catalan_word2vec_1GB.model'
    wv_model = KeyedVectors.load(WORD_EMBEDDING_FILE)

In [ ]:
REMAP_EMBEDDINGS: bool = True
USE_PRETRAINED: bool = True
MAX_LEN: int = 96

In [ ]:
from datasets import load_dataset
dataset = load_dataset("projecte-aina/sts-ca")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1491: FutureWarning: The repository for projecte-aina/sts-ca contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/projecte-aina/sts-ca
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
input_pairs = [(e["sentence1"], e["sentence2"], e["label"], ) for e in dataset["train"].to_list()]
input_pairs_val = [(e["sentence1"], e["sentence2"], e["label"], ) for e in dataset["validation"].to_list()]
input_pairs_test = [(e["sentence1"], e["sentence2"], e["label"], ) for e in dataset["test"].to_list()]

In [ ]:
all_input_pairs = input_pairs + input_pairs_val + input_pairs_test

sentences_1_preproc = [simple_preprocess(sentence_1) for sentence_1, _, _ in all_input_pairs]
sentences_2_preproc = [simple_preprocess(sentence_2) for _, sentence_2, _ in all_input_pairs]
sentence_pairs = list(zip(sentences_1_preproc, sentences_2_preproc))

sentences_pairs_flattened = sentences_1_preproc + sentences_2_preproc
diccionario = Dictionary(sentences_pairs_flattened)

In [ ]:
print("Max Len:", max([len(s) for s in sentences_1_preproc]), max([len(s) for s in sentences_2_preproc]))
print(list(diccionario.doc2idx(sentences_1_preproc[0])))
MAX_LEN = max([len(s) for s in sentences_2_preproc])
print(MAX_LEN)

Max Len: 30 30
[0, 11, 13, 1, 9, 10, 5, 14, 8, 7, 2, 8, 12, 2, 6, 4, 3, 15]
30


In [ ]:
def map_word_embeddings(
        sentence: str,
        sequence_len: int = MAX_LEN,
        fixed_dictionary: Optional[Dictionary] = None
) -> np.ndarray:
    """
    Map to word-embedding indices
    :param sentence:
    :param sequence_len:
    :param fixed_dictionary:
    :return:
    """
    sentence_preproc = simple_preprocess(sentence)[:sequence_len]
    _vectors = np.zeros(sequence_len, dtype=np.int32)
    index = 0
    for word in sentence_preproc:
        if fixed_dictionary is not None:
            if word in fixed_dictionary.token2id:
                # Sumo 1 porque el valor 0 está reservado a padding
                _vectors[index] = fixed_dictionary.token2id[word] + 1
                index += 1
        else:
            if word in wv_model.key_to_index:
                _vectors[index] = wv_model.key_to_index[word] + 1
                index += 1
    return _vectors


def map_pairs(
    sentence_pairs: List[Tuple[str, str, float]],
    sequence_len: int = MAX_LEN,
    fixed_dictionary: Optional[Dictionary] = None
) -> List[Tuple[Tuple[np.ndarray, np.ndarray], float]]:
    # Mapeo de los pares de oraciones a pares de vectores
    pares_vectores = []
    for i, (sentence_1, sentence_2, similitud) in enumerate(sentence_pairs):
        vector1 = map_word_embeddings(sentence_1, sequence_len, fixed_dictionary)
        vector2 = map_word_embeddings(sentence_2, sequence_len, fixed_dictionary)
        # Añadir a la lista
        pares_vectores.append(((vector1, vector2), similitud))
    return pares_vectores

In [ ]:
mapped_train = map_pairs(input_pairs, fixed_dictionary=diccionario if REMAP_EMBEDDINGS else None)
mapped_val = map_pairs(input_pairs_val, fixed_dictionary=diccionario if REMAP_EMBEDDINGS else None)
mapped_test = map_pairs(input_pairs_test, fixed_dictionary=diccionario if REMAP_EMBEDDINGS else None)

In [ ]:
mapped_train[0]

In [ ]:
# Define model 1
import tensorflow as tf
def model_1(
    input_length: int = MAX_LEN,
    dictionary_size: int = 10000000,
    embedding_size: int = 16,
    pretrained_weights: Optional[np.ndarray] = None,
    learning_rate: float = 1e-3,
    trainable: bool = False,
    use_cosine: bool = False,
) -> tf.keras.Model:
    # Input layers
    input_1 = tf.keras.Input(shape=(input_length,), dtype=tf.int32)
    input_2 = tf.keras.Input(shape=(input_length,), dtype=tf.int32)

    # Embedding layer
    if pretrained_weights is None:
        embedding = tf.keras.layers.Embedding(
            dictionary_size, embedding_size, input_length=input_length, mask_zero=True
        )
    else:
        dictionary_size = pretrained_weights.shape[0]
        embedding_size = pretrained_weights.shape[1]
        initializer = tf.keras.initializers.Constant(pretrained_weights)
        embedding = tf.keras.layers.Embedding(
            dictionary_size,
            embedding_size,
            input_length=input_length,
            mask_zero=True,
            embeddings_initializer=initializer,
            trainable=trainable,
        )

    # Apply embedding to input sequences
    embedded_1 = embedding(input_1)
    embedded_2 = embedding(input_2)

    # Global average pooling
    _input_mask_1, _input_mask_2 = tf.not_equal(input_1, 0), tf.not_equal(input_2, 0)
    pooled_1 = tf.keras.layers.GlobalAveragePooling1D()(embedded_1, mask=_input_mask_1)
    pooled_2 = tf.keras.layers.GlobalAveragePooling1D()(embedded_2, mask=_input_mask_2)

    # Compute similarity/distance
    if use_cosine:
        # Compute the cosine distance using a Lambda layer
        def cosine_distance(x):
            x1, x2 = x
            x1_normalized = tf.keras.backend.l2_normalize(x1, axis=1)
            x2_normalized = tf.keras.backend.l2_normalize(x2, axis=1)
            return 2.5 * (1.0 + tf.reduce_sum(x1_normalized * x2_normalized, axis=1))
        output = tf.keras.layers.Lambda(cosine_distance)([pooled_1, pooled_2])
    else:
        # Compute the cosine distance using a Lambda layer
        def normalized_product(x):
            x1, x2 = x
            x1_normalized = tf.keras.backend.l2_normalize(x1, axis=1)
            x2_normalized = tf.keras.backend.l2_normalize(x2, axis=1)
            return x1_normalized * x2_normalized

        output = tf.keras.layers.Lambda(normalized_product)([pooled_1, pooled_2])
        output = tf.keras.layers.Dropout(0.1)(output)
        output = tf.keras.layers.Dense(
            16,
            activation="relu",
        )(output)
        output = tf.keras.layers.Dropout(0.2)(output)
        output = tf.keras.layers.Dense(
            1,
            activation="sigmoid",
        )(output)

        output = tf.keras.layers.Lambda(lambda x: x * 5)(output)

    # Define the model
    model = tf.keras.Model(inputs=[input_1, input_2], outputs=output)

    # Compile the model
    model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate))

    return model

In [ ]:
from scipy.stats import pearsonr
def compute_pearson(x_, y_, model):
    # Get predictions for the model
    y_pred = model.predict(x_)
    # Compute pearson correlation
    correlation, _ = pearsonr(y_pred.flatten(), y_.flatten())
    return correlation

In [ ]:
# Definir constantes de entrenamiento
batch_size: int = 64
num_epochs: int = 32

In [ ]:
# Obtener x_train e y_train
def pair_list_to_x_y(pair_list: List[Tuple[Tuple[np.ndarray, np.ndarray], int]]) -> Tuple[Tuple[np.ndarray, np.ndarray], np.ndarray]:
    _x, _y = zip(*pair_list)
    _x_1, _x_2 = zip(*_x)
    return (np.row_stack(_x_1), np.row_stack(_x_2)), np.array(_y)

# Obtener las listas de train y test
x_train, y_train = pair_list_to_x_y(mapped_train)
x_val, y_val = pair_list_to_x_y(mapped_val)
x_test, y_test = pair_list_to_x_y(mapped_test)

In [ ]:
# Preparar los conjuntos de datos de entrenamiento y validación
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=len(x_train)).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.batch(batch_size)

In [ ]:
wv_model = word2vec_model

In [ ]:
_pretrained_weights: Optional[np.ndarray] = None
if USE_PRETRAINED:
    if REMAP_EMBEDDINGS:
        _pretrained_weights = np.zeros(
            (len(diccionario.token2id) + 1, wv_model.vector_size),  dtype=np.float32)
        for token, _id in diccionario.token2id.items():
            # Asegúrate de utilizar wv_model.wv para acceder a los vectores
            if token in wv_model.wv:
                _pretrained_weights[_id + 1] = wv_model.wv[token]
            else:
                # In W2V, OOV will not have a representation. We will use 0.
                pass
    else:
        # Not recommended (this will consume A LOT of RAM)
        _pretrained_weights = np.zeros((wv_model.wv.vectors.shape[0] + 1, wv_model.vector_size,),  dtype=np.float32)
        _pretrained_weights[1:, :] = wv_model.wv.vectors


In [ ]:
_pretrained_weights[:5,:]

In [ ]:
import pandas as pd

In [ ]:
resultats_models = pd.DataFrame(columns=['Model Name', 'Pearson_Train', 'Pearson_Val', 'Pearson_Test'])

In [ ]:
def afegir_resultats(model_name, Pearson_Train, Pearson_Val, Pearson_Test, dataframe):

    """
    Donat el nom d'un model i els seus resultats, els afegeix al DataFrame on es
    guarda la comparació de resultats.
    """

    new_row = pd.DataFrame({
        'Model Name': [model_name],
        'Pearson_Train': [Pearson_Train],
        'Pearson_Val': [Pearson_Val],
        'Pearson_Test': [Pearson_Test],
    })

    updated_dataframe = pd.concat([dataframe, new_row], ignore_index=True)
    return updated_dataframe

In [ ]:
def entrenament_model(nom_model, weights= None, entrenable= True, cosinus = False, resultats = resultats_models):
  model = model_1(pretrained_weights=weights, trainable=entrenable, use_cosine=cosinus, )
  model.summary()
  model.fit(train_dataset, epochs=16, validation_data=val_dataset)
  print(f"Correlació de Pearson (train): {compute_pearson(x_train, y_train, model)}")
  print(f"Correlació de Pearson (validation): {compute_pearson(x_val, y_val, model)}")
  print(f"Correlació de Pearson (test): {compute_pearson(x_test, y_test, model)}")
  resultats = afegir_resultats(nom_model,compute_pearson(x_train, y_train, model), compute_pearson(x_val, y_val, model), compute_pearson(x_test, y_test, model), resultats_models)
  return resultats

In [ ]:
resultats_models = entrenament_model("Word2Vec", weights= _pretrained_weights, entrenable =True, cosinus=False, resultats = resultats_models)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 30)]                 0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 30)]                 0         []                            
                                                                                                  
 embedding (Embedding)       (None, 30, 100)              1312600   ['input_1[0][0]',             
                                                                     'input_2[0][0]']             
                                                                                                  
 tf.math.not_equal (TFOpLam  (None, 30)                   0         ['input_1[0][0]']         

In [ ]:
resultats_models = entrenament_model("Word2Vec cosinus", weights= _pretrained_weights, entrenable =True, cosinus=True, resultats = resultats_models)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 30)]                 0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, 30)]                 0         []                            
                                                                                                  
 embedding_1 (Embedding)     (None, 30, 100)              1312600   ['input_3[0][0]',             
                                                                     'input_4[0][0]']             
                                                                                                  
 tf.math.not_equal_2 (TFOpL  (None, 30)                   0         ['input_3[0][0]']       

In [ ]:
for _ in range(5):
  resultats_models = entrenament_model("Random", weights=None, entrenable =True, cosinus=False, resultats = resultats_models)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 30)]                 0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, 30)]                 0         []                            
                                                                                                  
 embedding_1 (Embedding)     (None, 30, 16)               1600000   ['input_3[0][0]',             
                                                          00         'input_4[0][0]']             
                                                                                                  
 tf.math.not_equal_2 (TFOpL  (None, 30)                   0         ['input_3[0][0]']       

In [ ]:
resultats_models = entrenament_model("Random", weights=None, entrenable =True, cosinus=False, resultats = resultats_models)

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 30)]                 0         []                            
                                                                                                  
 input_10 (InputLayer)       [(None, 30)]                 0         []                            
                                                                                                  
 embedding_4 (Embedding)     (None, 30, 16)               1600000   ['input_9[0][0]',             
                                                          00         'input_10[0][0]']            
                                                                                                  
 tf.math.not_equal_8 (TFOpL  (None, 30)                   0         ['input_9[0][0]']       

In [ ]:
resultats_models = entrenament_model("Random cosinus", weights= None, entrenable =True, cosinus=True, resultats = resultats_models)

In [ ]:
resultats_models